# View Campaign and Interactions

In the first notebook `Personalize_BuildCampaign.ipynb` you successfully built and deployed a recommendation model using deep learning with Amazon Personalize.

This notebook will expand on that and will walk you through adding the ability to react to real time behavior of users. If their intent changes while browsing a movie, you will see revised recommendations based on that behavior.

It will also showcase demo code for simulating user behavior selecting movies before the recommendations are returned.

Below we start with just importing libraries that we need to interact with Personalize

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

Below you will paste in the campaign ARN that you used in your previous notebook. Also pick a random user ID from 50 - 300. 

Lastly you will also need to find your Dataset Group ARN from the previous notebook.

In [2]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
HRNN_Campaign_ARN = "arn:aws:personalize:us-east-1:930444659029:campaign/DEMO-campaign"

# Define User 
USER_ID = "676"

# Dataset Group Arn:
datasetGroupArn = "arn:aws:personalize:us-east-1:930444659029:dataset-group/DEMO-dataset-group"

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## Creating an Event Tracker

Before your recommendation system can respond to real time events you will need an event tracker, the code below will generate one and can be used going forward with this lab. Feel free to name it something more clever.

In [4]:
response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=datasetGroupArn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

arn:aws:personalize:us-east-1:930444659029:event-tracker/bbe80586
b8a5944c-8095-40ff-a915-2a6af53b7f55


## Configuring Source Data

Above you'll see your tracking ID and this has been assigned to a variable so no further action is needed by you. The lines below are going to setup the data used for recommendations so you can render the list of movies later.

In [5]:
data = pd.read_csv('./ml-20m/ratings.csv', sep=',', dtype={'userid': "int64", 'movieid': "int64", 'rating': "float64", 'timestamp': "int64"})
pd.set_option('display.max_rows', 5)
data.rename(columns = {'userId':'USER_ID','movieId':'ITEM_ID','rating':'RATING','timestamp':'TIMESTAMP'}, inplace = True)
data = data[data['RATING'] > 3]                # keep only movies rated 3
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data

,USER_ID,ITEM_ID,TIMESTAMP
0,1,2,1112486027
1,1,29,1112484676
...,...,...,...
20000259,138493,69526,1259865108
20000261,138493,70286,1258126944


In [6]:
items = pd.read_csv('./ml-20m/movies.csv', sep=',', usecols=[0,1], header=0)
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

items

USER: 40094
ITEM: Hotel Rwanda (2004)


,ITEM_ID,TITLE
0,1,Toy Story (1995)
1,2,Jumanji (1995)
...,...,...
27276,131260,Rentun Ruusu (2001)
27277,131262,Innocence (2014)


## Getting Recommendations

Just like in the previous notebook it is a great idea to get a list of recommendatiosn first and then see how additional behavior by a user alters the recommendations.

In [7]:
# Get Recommendations as is
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = HRNN_Campaign_ARN,
    userId = USER_ID,
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))
print(item_list)

Recommendations: [
  "Signs (2002)",
  "Panic Room (2002)",
  "Vanilla Sky (2001)",
  "American Pie 2 (2001)",
  "Blade II (2002)",
  "Bourne Identity, The (2002)",
  "Star Wars: Episode II - Attack of the Clones (2002)",
  "Memento (2000)",
  "Fast and the Furious, The (2001)",
  "Unbreakable (2000)",
  "Snatch (2000)",
  "Austin Powers in Goldmember (2002)",
  "Resident Evil (2002)",
  "xXx (2002)",
  "Sum of All Fears, The (2002)",
  "Others, The (2001)",
  "American Beauty (1999)",
  "Pulp Fiction (1994)",
  "Spider-Man (2002)",
  "Minority Report (2002)",
  "Rock, The (1996)",
  "Ring, The (2002)",
  "Black Hawk Down (2001)",
  "Ocean's Eleven (2001)",
  "Schindler's List (1993)"
]
[{'itemId': '5502'}, {'itemId': '5266'}, {'itemId': '4975'}, {'itemId': '4718'}, {'itemId': '5254'}, {'itemId': '5418'}, {'itemId': '5378'}, {'itemId': '4226'}, {'itemId': '4369'}, {'itemId': '3994'}, {'itemId': '4011'}, {'itemId': '5481'}, {'itemId': '5219'}, {'itemId': '5507'}, {'itemId': '5400'}, {'i

## Simulating User Behavior

The lines below provide a code sample that simulates a user interacting with a particular item, you will then get recommendations that differ from those when you started.

In [8]:
session_dict = {}

In [9]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
)

Immediately below this line will update the tracker as if the user has clicked a particular title.

In [10]:
# Pick a movie, we will use ID 1653 or Gattica
send_movie_click(USER_ID=USER_ID, ITEM_ID=1653)

After executing this block you will see the alterations in the recommendations now that you have event tracking enabled and that you have sent the events to the service.

In [11]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = HRNN_Campaign_ARN,
    userId = str(USER_ID),
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))
print(item_list)

Recommendations: [
  "Signs (2002)",
  "Fifth Element, The (1997)",
  "Gattaca (1997)",
  "Unbreakable (2000)",
  "Face/Off (1997)",
  "Predator (1987)",
  "Dark City (1998)",
  "Star Wars: Episode II - Attack of the Clones (2002)",
  "Cube (1997)",
  "Spider-Man (2002)",
  "Game, The (1997)",
  "Minority Report (2002)",
  "X-Files: Fight the Future, The (1998)",
  "Twelve Monkeys (a.k.a. 12 Monkeys) (1995)",
  "Rock, The (1996)",
  "Vanilla Sky (2001)",
  "Starship Troopers (1997)",
  "Bourne Identity, The (2002)",
  "Sneakers (1992)",
  "American Beauty (1999)",
  "Austin Powers in Goldmember (2002)",
  "Memento (2000)",
  "Pulp Fiction (1994)",
  "X-Men (2000)",
  "Star Wars: Episode I - The Phantom Menace (1999)"
]
[{'itemId': '5502'}, {'itemId': '1527'}, {'itemId': '1653'}, {'itemId': '3994'}, {'itemId': '1573'}, {'itemId': '3527'}, {'itemId': '1748'}, {'itemId': '5378'}, {'itemId': '2232'}, {'itemId': '5349'}, {'itemId': '1625'}, {'itemId': '5445'}, {'itemId': '1909'}, {'itemId':

## Conclusion

You can see now that recommendations are altered by changing the movie that a user interacts with, this system can be modified to any application where users are interacting with a collection of items. These tools are available at any time to pull down and start exploring what is possible with the data you have.

Finally when you are ready to remove the items from your account, open the `Cleanup.ipynb` notebook and execute the steps there.


In [ ]:
eventTrackerArn = response['eventTrackerArn']
print("Tracker ARN is: " + str(eventTrackerArn))